In [2]:
import sys 
sys.path.insert(0, '/home/yikuan/project/Code/')

In [4]:
import pandas as pd 
import numpy as np

In [1]:
file_config = {
#     'vocab':'/home/shared/01_data/01_dictionary/pureicd_vocab_diag',
    'train': '/home/shared/yikuan/ACM/data/Depression/Depression_train.parquet/',
    'test': '/home/shared/yikuan/ACM/data/Depression/Depression_test.parquet/',
}
params = {
    'code': 'code'
}


In [5]:
data = pd.read_parquet(file_config['train'])
tdata = pd.read_parquet(file_config['test'])
total = pd.concat([data, tdata]).reset_index(drop=True)

In [6]:
filteringDict = {
    'deltaAge': 36,
    'numSep': 10,
    'unique_leng':10,
#     'tfidf-threshold':1.5,
    'months': 12
}

In [7]:
total['numSep'] = total[params['code']].apply(lambda x: len(np.where(x=='SEP')[0]))
print('begining: number of patient with label 0 and 1: {},{}'.format(len(total[(total.label==0)]),len(total[(total.label==1)])))

# keep patient with number of visits larger than numSep
total = total[(total.numSep>filteringDict['numSep'])]
total = total.reset_index(drop=True)
print('filtering by visit: number of patient with label 0 and 1: {},{}'.format(len(total[(total.label==0)]),len(total[(total.label==1)])))

# keep patient with records having time span larger than deltaAge
total['deltaAge'] = total.age.apply(lambda x: int(x[-1]) - int(x[0]))
total = total[total['deltaAge'] >filteringDict['deltaAge']]
total = total.reset_index(drop=True)
print('filtering by time span: number of patient with label 0 and 1: {},{}'.format(len(total[(total.label==0)]),len(total[(total.label==1)])))

# keep patient with number of unique code larger than unique_leng
total['unique_leng'] = total['code'].apply(lambda x: len(np.unique(x)))
total = total[total['unique_leng'] >filteringDict['unique_leng']]
total = total.reset_index(drop=True)
print('filtering by number of unique codes: number of patient with label 0 and 1: {},{}'.format(len(total[(total.label==0)]),len(total[(total.label==1)])))

begining: number of patient with label 0 and 1: 2633645,386961
filtering by visit: number of patient with label 0 and 1: 1692397,313203
filtering by time span: number of patient with label 0 and 1: 1379731,248345
filtering by number of unique codes: number of patient with label 0 and 1: 1221749,235048


In [8]:
# split
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(total, test_size=0.3, random_state=42)
X_train.to_parquet('/home/shared/yikuan/ACM/data/Depression/Depression_clean_train.parquet')
X_test.to_parquet('/home/shared/yikuan/ACM/data/Depression/Depression_clean_test.parquet')